## Hello World SC
#### 07.0 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-09


## Setup
See notebook 04.1, loading `algo_util.py`, the five accounts and the Purestake credentials
* Consider hiding this code

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json
import base64
import datetime

In [3]:
from pyteal import *

In [4]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 19847679


In [5]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

ADLR27NB3V62INPRGCK63YMRWQCU4J2OE6PTIGZSZSXZNO5KSLAU7TPSFU
FEJNBD5DI3TC53AZNGSUAXAR6HAELQF4PLAHSVDUG3EP5WXEKRVULIWOJE
AH3NJBIMCZZLXY7NR2NENBWUS4P63JOJFUYM2O3DTW4J5E2ENQL5ISFGKU


### Some helper functions

In [6]:
def read_local_state(client, addr, app_id):
    # reads a user's local state
    # client = algod_client
    # addr = public addr of the user that we want to inspect
    results = client.account_info(addr)
    for local_state in results["apps-local-state"]:
        if local_state["id"] == app_id:
            if "key-value" not in local_state:
                return {}
            return format_state(local_state["key-value"])
    return {}

def read_global_state(client, app_id):
    # reads an app's global state
    return  algod_client.application_info(app_id)["params"]["global-state"]

def format_state(state):
    # formats the state (local/global) nicely 
    formatted = {}
    textvariables = {'Info','Note'}        # <---- update this! (List of Text variables in SC)
    for item in state:
        key = base64.b64decode(item["key"]).decode("utf-8")
        value = item["value"]
        if value["type"] == 1:
            if key in textvariables:                 # Format text variables
                formatted_value = base64.b64decode(value["bytes"]).decode("utf-8")
            else:                                    # Format addresses
                formatted_value = base64.b32encode(base64.b64decode(value["bytes"]))
            formatted[key] = formatted_value
        else:
            formatted[key] = value["uint"]
    return formatted

## A first stateful smart contract
The stateful smart contract consists of two parts
* The `approval_program` that handles everything except opt out
* The `clear_state_program` that handles the opt out

## Our first project: HelloWorld Local
* Idea: just write "Hello World" into the global state
* AND into the global state of every user
* Do not do anything else

#### Step 1: Define Approval program
This is the program that handles all interactions except opt out:
* Creation of the SC `handle_creation`
* Opt-in of individual users `handle_register`
* Interactions (calls) of individual users `handle_interact`


In [7]:
handle_creation = Seq(
    [
        # Initialize a GLOBAL variable
        App.globalPut(Bytes("Note"),       Bytes("Hello world!")),
        # Return "OK"
        Return(Int(1))        
    ]
)

handle_optin = Seq(
    [
        # Initialize a LOCAL variable
        # Int(0) = current user
        App.localPut(Int(0), Bytes("Note"), Bytes("Welcome on board!")),
        # Return "OK"
        Return(Int(1))        
    ]
)


handle_closeout = Return( Int(1) )                # Not doing anything, returning "OK"  

handle_updateapp = Return( Int(0) )               # Always FALSE ... updating not allowed

handle_deleteapp = Return(
    Txn.sender() == Global.creator_address()    # only TRUE if delete request is made by creator
)

# handle interaction
handle_noop  = Return ( Int(1) )                   # Not doing anything, returning "OK"           



In [8]:
hello_approval_pyteal = Cond(
    [Txn.application_id() == Int(0),              handle_creation],
    [Txn.on_completion()  == OnComplete.OptIn,    handle_optin],
    [Txn.on_completion()  == OnComplete.CloseOut, handle_closeout],
    [Txn.on_completion()  == OnComplete.UpdateApplication, handle_updateapp],
    [Txn.on_completion()  == OnComplete.DeleteApplication, handle_deleteapp],
    [Txn.on_completion()  == OnComplete.NoOp, handle_noop],
)

#### Compile PyTEAL -> TEAL
* Notice the `Mode.Application` (was `Mode.Signature`)

In [9]:
hello_approval_teal = compileTeal(hello_approval_pyteal,mode=Mode.Application, version=3)
print(hello_approval_teal)

#pragma version 3
txn ApplicationID
int 0
==
bnz l12
txn OnCompletion
int OptIn
==
bnz l11
txn OnCompletion
int CloseOut
==
bnz l10
txn OnCompletion
int UpdateApplication
==
bnz l9
txn OnCompletion
int DeleteApplication
==
bnz l8
txn OnCompletion
int NoOp
==
bnz l7
err
l7:
int 1
return
l8:
txn Sender
global CreatorAddress
==
return
l9:
int 0
return
l10:
int 1
return
l11:
int 0
byte "Note"
byte "Welcome on board!"
app_local_put
int 1
return
l12:
byte "Note"
byte "Hello world!"
app_global_put
int 1
return


#### Step 1b: Define Clear State program
* This program handles forced opt-outs

In [10]:
hello_clear_pyteal =  Return(Int(1))    # not doing anything

In [11]:
hello_clear_teal = compileTeal(hello_clear_pyteal,mode=Mode.Application, version=3)
print(hello_clear_teal)

#pragma version 3
int 1
return


#### Compile TEAL -> Bytecode
This is slightly different ... we need one additional step for Byte-encoding

In [12]:
hello_approval_b64 = algod_client.compile(hello_approval_teal)
Hello_Approval =  base64.b64decode(hello_approval_b64['result'])

hello_clear_b64 = algod_client.compile(hello_clear_teal)
Hello_Clear =  base64.b64decode(hello_clear_b64['result'])

## Deploy Smart Contract

##### Bob deploys the smart contract
* Reserve global storage with `StateSchema`
* New command `ApplicationCreateTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationCreateTxn)

In [13]:
# Step 1: Prepare the transaction
sp = algod_client.suggested_params()

# How much space do we need?
global_ints = 0    # no numeric variable
global_bytes = 1   # for  Note
hello_global_schema = transaction.StateSchema(global_ints, global_bytes)

local_ints = 0     # No local variables
local_bytes = 1    # for Note
hello_local_schema = transaction.StateSchema(local_ints, local_bytes)

txn = transaction.ApplicationCreateTxn(
      sender = Bob['public'],              # <-- sender public
      sp = sp,                             # <-- sp
      on_complete = 0,                     # <-- when finished do nothing
      approval_program = Hello_Approval,   # <-- approval program 
      clear_program = Hello_Clear,         # <-- clear program 
      global_schema = hello_global_schema, # <-- reserve global space 
      local_schema = hello_local_schema    # <-- reserve local space
    )

In [14]:
# Step 2: sign transaction
stxn = txn.sign(Bob['private'])

# Step 3: send
txid=algod_client.send_transactions([stxn])

# Step 4: wait for ...
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19847765.
Waiting for round 19847765 to finish.
Waiting for round 19847766 to finish.
Transaction M5F4URIG2WGFCPU4VRLWDSRWQZLD6ALRCP45O532KMCVAYP3NBKA confirmed in round 19847767.


In [16]:
app_id = txinfo["application-index"]
print("Created new app-id:", app_id)

Created new app-id: 71734153


## The Smart Contract is now deployed
* And there is alreasdy something to see

In [17]:
format_state(read_global_state(algod_client,app_id))

{'Note': 'Hello world!'}

In [19]:
# Program code immediately visible on the web
print('https://testnet.algoexplorer.io/application/{}'.format(app_id))

https://testnet.algoexplorer.io/application/71734153


## Using the Smart Contract (1): Users opt-in
* Nothing will happen, but we can test everything
* Using `ApplicationOptInTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationOptInTxn)

In [27]:
user = Alice

# Step 1: prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationOptInTxn(user['public'], sp, app_id)

# Step 2: sign transaction
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: await confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19847797.
Waiting for round 19847797 to finish.
Waiting for round 19847798 to finish.
Transaction 7MV77G2BAICIUIXACIY2KOYHDAHD2WBTPTWX35UCF3DSLY2ZPJQA confirmed in round 19847799.


#### Watch the state of the contract evolve

In [28]:
format_state(read_global_state(algod_client,app_id))

{'Note': 'Hello world!'}

In [32]:
read_local_state(algod_client,user['public'],app_id)

{}

## Users can call the Smart contract ("make a visit")
* Strangley, calling a Smart Contact is done with a `ApplicationNoOpTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationNoOpTxn)

In [33]:
user = Bob

# Step 1: prepare
sp = algod_client.suggested_params()
txn = transaction.ApplicationNoOpTxn(user['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: wait for condfirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19847809.
Waiting for round 19847809 to finish.
Waiting for round 19847810 to finish.
Transaction EB6SWXP2BWZ7BIMXPHWMGM2HMWEXNC4CMZZJVZE6NMGSMQOHEIVQ confirmed in round 19847811.


#### The state does not change

In [34]:
format_state(read_global_state(algod_client,app_id))

{'Note': 'Hello world!'}

In [35]:
read_local_state(algod_client,user['public'],app_id)

{}

## Users close out (leave) App
* With a `ApplicationCloseOutTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationCloseOutTxn)

In [36]:
user = Alice

# Step 1: prepare
sp = algod_client.suggested_params()
txn = transaction.ApplicationCloseOutTxn(user['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: wait for condfirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19847815.
Waiting for round 19847815 to finish.
Waiting for round 19847816 to finish.
Transaction 67WEQRC4OHIMDEEPNFNIJAT7M3QDHRGPO63JU6OK3U7OPRWL4TVA confirmed in round 19847817.


## Deleting the app
* Rather important, because an address can only create **10 apps**
* App can be deleted by creator

In [37]:
creator = Bob

# Step 1: Prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationDeleteTxn(creator['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(creator['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step4 : wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

# display results
print("Deleted app-id:", txinfo["txn"]["txn"]["apid"])

Current round is  19847820.
Waiting for round 19847820 to finish.
Waiting for round 19847821 to finish.
Transaction MI54BNKROCL4PKBDEXRNJL4YV6QEPTFCVAX6TVLWIVGJK2KYYAUQ confirmed in round 19847822.
Deleted app-id: 71734153
